# First MSSP-PD(NQP) example

In [1]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the synthetic instance of Section 3.2

In [2]:
nodes_number = 25
agents_number = 4
synthetic_5x5_df = pd.read_csv("data/d_it_ij_5x5_1it.csv",
                               header=0,
                               names=["i", "j", "d_ij"])

# convert d_ij column into float
# print(synthetic_5x5_df.dtypes)
synthetic_5x5_df["d_ij"] = synthetic_5x5_df["d_ij"].str.replace(
    ",", ".").astype(np.float64)
# print(synthetic_5x5_df.dtypes)

# counting network nodes starting from 0
synthetic_5x5_df[["i", "j"]] = synthetic_5x5_df[["i", "j"]] - 1


Set up datas that will be used to solve the problem

In [3]:
Arc = namedtuple("Arc", synthetic_5x5_df.columns)
arcs = [Arc(*row) for row in synthetic_5x5_df.itertuples(index=False)]  # as List to avoid nusty bugs

nodes = np.unique(np.concatenate((synthetic_5x5_df["i"].to_numpy(),
                                  synthetic_5x5_df["j"].to_numpy())))
agent_sources = np.array([0, 2, 3, 4])
agent_terminus = np.array([20, 22, 23, 24])

agents = range(agents_number)
# D = synthetic_5x5_df["d_ij"].to_numpy()

## Manage the problem

Create the problem

In [4]:
MSPP_PD_NQP_pb = gb.Model("First MSPP_PD_NQP_pb")
MSPP_PD_NQP_pb.setParam("OutputFlag", 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


Define decision variables

In [5]:
X_var_shape = len(nodes), len(nodes), agents_number

R_var_shape = len(nodes), agents_number

W_var_shape = len(nodes), agents_number, agents_number


X = MSPP_PD_NQP_pb.addMVar(X_var_shape,
                           vtype=GRB.BINARY,  # 5) Binary constraints
                           name="k-th agent traverse arc (i,j)")

R = MSPP_PD_NQP_pb.addMVar(R_var_shape,
                           vtype=GRB.BINARY,  # 13) Binary constraints
                           name="more agents traverse node i")

W = MSPP_PD_NQP_pb.addMVar(W_var_shape,
                           vtype=GRB.BINARY,  # 33) Non-negativity constraints
                           name="agents k and k_ both traverse node i")


Define the objective functions

In [6]:
# 1-3, 28) Objective function (linearized version)
distance_objn = gb.quicksum(
    arc.d_ij*X[arc.i, arc.j, k]
    for arc in arcs for k in agents
)
penalties_objn = gb.quicksum(
    W[i, k, k_] for i in nodes for k in agents for k_ in agents if k_ < k
)

w_p = 0.5
w_d = 1-w_p
MSPP_PD_NQP_pb.setObjectiveN(distance_objn, index=0, weight=w_d,
                             name="Distance")
MSPP_PD_NQP_pb.setObjectiveN(penalties_objn, index=1, weight=w_p,
                             name="Penalty")


MSPP_PD_NQP_pb.ModelSense = GRB.MINIMIZE

Add constraints

In [7]:
# 4) Flow constraints

def compute_flow(X, node, arcs, agent):
    flow_out = gb.quicksum(X[node, arc.j, agent]
                           for arc in arcs if arc.i == node)
    flow_in = gb.quicksum(X[arc.i, node, agent]
                          for arc in arcs if arc.j == node)
    return flow_out - flow_in


for k in agents:
    for i in nodes:
        if i == agent_sources[k]:
            MSPP_PD_NQP_pb.addConstr(compute_flow(X, i, arcs, k) == 1,
                                     name=f"Flow constr related to agent {k} in node {i}")
        elif i == agent_terminus[k]:
            MSPP_PD_NQP_pb.addConstr(compute_flow(X, i, arcs, k) == -1,
                                     name=f"Flow constr related to agent {k} in node {i}")
        else:
            MSPP_PD_NQP_pb.addConstr(compute_flow(X, i, arcs, k) == 0,
                                     name=f"Flow constr related to agent {k} in node {i}")


In [8]:
# 10,11) Turning on r_i constraints

for k in agents:
    for node in nodes:
        MSPP_PD_NQP_pb.addConstr(
            R[node, k] >= gb.quicksum(X[node, arc.j, k]
                                      for arc in arcs if arc.i == node)
        )
        MSPP_PD_NQP_pb.addConstr(
            R[node, k] >= gb.quicksum(X[arc.i, node, k]
                                      for arc in arcs if arc.j == node)
        )


In [9]:
# 29) Turning off r_i constraints

for k in agents:
    for node in nodes:
        MSPP_PD_NQP_pb.addConstr(
            R[node, k] <= (
                gb.quicksum(X[node, arc.j, k] for arc in arcs if arc.i == node) +
                gb.quicksum(X[arc.i, node, k] for arc in arcs if arc.j == node)
            )
        )


In [10]:
# 30-32) Well-defined W variable

for node in nodes:
    for k in agents:
        for k_ in agents:
            if k_ < k:
                MSPP_PD_NQP_pb.addConstr(
                    W[node, k, k_] <= R[node, k]
                )
                MSPP_PD_NQP_pb.addConstr(
                    W[node, k, k_] <= R[node, k_]
                )
                MSPP_PD_NQP_pb.addConstr(
                    W[node, k, k_] >= R[node, k] + R[node, k_] - 1
                )


Solve the problem

In [11]:
MSPP_PD_NQP_pb.optimize()

Report results

In [12]:
print("Result of the optimization is:")
if MSPP_PD_NQP_pb.Status == 2:
    print("optimal")
elif MSPP_PD_NQP_pb.Status == 3:
    print("infeasible")
elif MSPP_PD_NQP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")


Result of the optimization is:
optimal


In [13]:
n_objectives = MSPP_PD_NQP_pb.NumObj
n_solutions = MSPP_PD_NQP_pb.SolCount

print(f"The optimization founds {n_solutions} solutions:")
for sol_n in range(n_solutions):
    MSPP_PD_NQP_pb.params.SolutionNumber = sol_n

    print(f"Solution {sol_n}:", end="")
    obj_tot_value = 0
    for obj_n in range(n_objectives):
        MSPP_PD_NQP_pb.params.ObjNumber = obj_n
        obj_tot_value = obj_tot_value + MSPP_PD_NQP_pb.ObjNWeight*MSPP_PD_NQP_pb.ObjNVal
        print(f" {MSPP_PD_NQP_pb.ObjNName}={MSPP_PD_NQP_pb.ObjNVal} ", end="|")
    print(f" Weighted Total={obj_tot_value}")

    for k in agents:
        print(f"Agent {k} will follow the path:")
        for arc in arcs:
            if math.isclose(X.X[arc.i, arc.j, k], 1):
                print(f"{arc.i}->{arc.j}", end="\t")
        print()

    print()

The optimization founds 4 solutions:
Solution 0: Distance=17.0 | Penalty=1.0 | Weighted Total=9.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->9	9->13	13->19	19->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	

Solution 1: Distance=19.0 | Penalty=3.0 | Weighted Total=11.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->9	9->13	13->19	19->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	

Solution 2: Distance=24.0 | Penalty=0.0 | Weighted Total=12.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->9	9->13	13->19	19->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	

Solution 3: Distance=25.0 | Penalty=4.0 | Weighted Total=14.5
Agent 0 will follow the path:
0->6	6->12	

## Explore Pareto solutions

Define some useful functions

In [14]:
def evaluate_NQP_distance_and_penalty(X_values, arcs):

    def set_r_ik(X, R, arc, agent):
        if math.isclose(X[arc.i, arc.j, agent], 1):
            R[arc.i, agent] = 1
            R[arc.j, agent] = 1

    distance_obj = 0
    penalty_obj = 0
    num_nodes = X_values.shape[0]
    num_agents = X_values.shape[-1]

    R = np.zeros((num_nodes, num_agents))

    for k in range(num_agents):
        for arc in arcs:
            distance_obj += arc.d_ij*X_values[arc.i, arc.j, k]
            set_r_ik(X_values, R, arc, k)

    for i in range(num_nodes):
        for k in range(num_agents):
            for k_ in range(k):  # k_<k:
                penalty_obj += R[i,k]*R[i,k_]

    return distance_obj, penalty_obj


def path_for_agent(X, k):

    return [arc for arc in arcs if math.isclose(X.x[arc.i, arc.j, k], 1)]


def get_opt_agents_paths(X):

    return {k: path_for_agent(X, k) for k in agents}


def print_path(path):

    for arc in path:
        print(f"{arc.i}->{arc.j}", end="\t")
    print()


Create the range of weights to test

In [15]:
w_p_start = 0.01
w_p_stop = 1
delta = 0.01

w_p_range = np.arange(w_p_start, w_p_stop, delta)

Compute solutions to MSPP-PD(NQP) for different weights

In [16]:
opt_distance_values = []
opt_penalty_values = []
prev_opt_distance, prev_opt_penalty = math.nan, math.nan
opt_agents_paths = []
# Optimal solution will remain the same for a certain interval of weights of the 2 objectives 
w_p_intervals_start = []


for w_p in w_p_range:

    w_d = 1 - w_p

    MSPP_PD_NQP_pb.reset()

    # Change weights of the 2 objectives
    MSPP_PD_NQP_pb.setObjectiveN(
        distance_objn, index=0, weight=w_d, name="Distance")
    MSPP_PD_NQP_pb.setObjectiveN(
        penalties_objn, index=1, weight=w_p, name="Penalty")

    MSPP_PD_NQP_pb.optimize()
    opt_distance, opt_penalty = evaluate_NQP_distance_and_penalty(X.x, arcs)

    if not math.isclose(opt_distance, prev_opt_distance) or not math.isclose(opt_penalty, prev_opt_penalty):

        opt_distance_values.append(opt_distance)
        opt_penalty_values.append(opt_penalty)
        opt_agents_paths.append(get_opt_agents_paths(X))
        w_p_intervals_start.append(w_p)

        prev_opt_distance, prev_opt_penalty = opt_distance, opt_penalty

Compute the end of the intervals in which solution does not change

In [17]:
w_p_intervals_start.append(w_p_stop)
w_p_intervals_stop = [ w_p - delta for w_p in w_p_intervals_start ]
del w_p_intervals_start[-1], w_p_intervals_stop[0]
w_p_intervals = list(zip(w_p_intervals_start, w_p_intervals_stop))

Report results

In [18]:
pareto_results_df = pd.DataFrame(
    {
        "w_p_interval": w_p_intervals,
        "Optimal distance": opt_distance_values,
        "Optimal penalty": opt_penalty_values
    }
)
pareto_results_df

,w_p_interval,Optimal distance,Optimal penalty
0,"(0.01, 0.2)",16.0,5.0
1,"(0.21000000000000002, 0.54)",17.0,1.0
2,"(0.55, 0.99)",18.2,0.0


In [19]:
for i in pareto_results_df.index:
    print(f"Solution for w_p in {w_p_intervals[i]}:")

    for k, path in opt_agents_paths[i].items():
        print(f"Agent {k} will follow the path:")
        print_path(path)

    print()


Solution for w_p in (0.01, 0.2):
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->7	7->12	12->18	18->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	

Solution for w_p in (0.21000000000000002, 0.54):
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->9	9->13	13->19	19->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	

Solution for w_p in (0.55, 0.99):
Agent 0 will follow the path:
0->6	6->11	11->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->17	17->22	
Agent 2 will follow the path:
3->9	9->13	13->19	19->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	



### About different solutions

We can notice that the solution reported in the article for $w_p\in[0.01, 0.2]$ (Fig. 5) is diferent from what we found.  
However our solution is still an optimal solution since it has the same distance and penalty (therefore same total objective value) of the paper's solution. We'll show this in the following...

Set weights as in the first interval

In [20]:
w_p = 0.01
w_d = 1 - w_p

# Set objectives weights
MSPP_PD_NQP_pb.reset()

MSPP_PD_NQP_pb.setObjectiveN(
    distance_objn, index=0, weight=w_d, name="Distance")
MSPP_PD_NQP_pb.setObjectiveN(
    penalties_objn, index=1, weight=w_p, name="Penalty")

Force the solution to be the same as in Fig. 5

In [21]:
MSPP_PD_NQP_pb.addConstr(X[0,6,0] == 1)
MSPP_PD_NQP_pb.addConstr(X[6,12,0] == 1)
MSPP_PD_NQP_pb.addConstr(X[12,16,0] == 1)
MSPP_PD_NQP_pb.addConstr(X[16,20,0] == 1)

MSPP_PD_NQP_pb.addConstr(X[2,7,1] == 1)
MSPP_PD_NQP_pb.addConstr(X[7,12,1] == 1)
MSPP_PD_NQP_pb.addConstr(X[12,16,1] == 1)
MSPP_PD_NQP_pb.addConstr(X[16,22,1] == 1)

MSPP_PD_NQP_pb.addConstr(X[3,7,2] == 1)
MSPP_PD_NQP_pb.addConstr(X[7,12,2] == 1)
MSPP_PD_NQP_pb.addConstr(X[12,17,2] == 1)
MSPP_PD_NQP_pb.addConstr(X[17,23,2] == 1)

MSPP_PD_NQP_pb.addConstr(X[4,8,3] == 1)
MSPP_PD_NQP_pb.addConstr(X[8,14,3] == 1)
MSPP_PD_NQP_pb.addConstr(X[14,18,3] == 1)
MSPP_PD_NQP_pb.addConstr(X[18,24,3] == 1)

<MConstr () *awaiting model update*>

Solve the problem

In [22]:
MSPP_PD_NQP_pb.optimize()

Report results

In [23]:
print("Result of the optimization is:")
if MSPP_PD_NQP_pb.Status == 2:
    print("optimal")
elif MSPP_PD_NQP_pb.Status == 3:
    print("infeasible")
elif MSPP_PD_NQP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")

Result of the optimization is:
optimal


In [24]:
n_objectives = MSPP_PD_NQP_pb.NumObj
n_solutions = MSPP_PD_NQP_pb.SolCount

print(f"The optimization founds {n_solutions} solutions:")
for sol_n in range(n_solutions):
    MSPP_PD_NQP_pb.params.SolutionNumber = sol_n

    print(f"Solution {sol_n}:", end="")
    obj_tot_value = 0
    for obj_n in range(n_objectives):
        MSPP_PD_NQP_pb.params.ObjNumber = obj_n
        obj_tot_value = obj_tot_value + MSPP_PD_NQP_pb.ObjNWeight*MSPP_PD_NQP_pb.ObjNVal
        print(f" {MSPP_PD_NQP_pb.ObjNName}={MSPP_PD_NQP_pb.ObjNVal} ", end="|")
    print(f" Weighted Total={obj_tot_value}")

    for k in agents:
        print(f"Agent {k} will follow the path:")
        for arc in arcs:
            if math.isclose(X.X[arc.i, arc.j, k], 1):
                print(f"{arc.i}->{arc.j}", end="\t")
        print()

    print()

The optimization founds 1 solutions:
Solution 0: Distance=16.0 | Penalty=5.0 | Weighted Total=15.89
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	
Agent 1 will follow the path:
2->7	7->12	12->16	16->22	
Agent 2 will follow the path:
3->7	7->12	12->17	17->23	
Agent 3 will follow the path:
4->8	8->14	14->18	18->24	



So we can state that the solution to the MSPP-PD(NQP) for the considered network is not unique for $w_p\in[0.01, 0.2]$ since
the solution found by us and the one reported in the article have the same objective value